In [ ]:
import os
from sklearn.model_selection import train_test_split
import pandas as pd
import mlflow
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder, 
    SplineTransformer, 
    QuantileTransformer, 
    RobustScaler,
    PolynomialFeatures,
    KBinsDiscretizer,
)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score, f1_score, precision_score, recall_score, 
    log_loss, confusion_matrix, mean_squared_error, r2_score
)
from catboost import CatBoostClassifier
from dotenv import load_dotenv, find_dotenv
import psycopg
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# подгружаем .env
load_dotenv()

In [ ]:
TABLE_NAME = "alt_users_churn" # таблица с данными в postgres 
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"

# Параметры для трекинга эксперимента
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = "churn_experiment_imartnv"
RUN_NAME = "preprocessing" 
REGISTRY_MODEL_NAME = "churn_model_martynov_alexey"

# Настрофка отображения
pd.options.display.max_columns = 100
pd.options.display.max_rows = 64

sns.set_style("white")
sns.set_theme(style="whitegrid") 

In [ ]:
# Подключение к базе и получение данных
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST"),
    "port": os.getenv("DB_DESTINATION_PORT"),
    "dbname": os.getenv("DB_DESTINATION_NAME"),
    "user": os.getenv("DB_DESTINATION_USER"),
    "password": os.getenv("DB_DESTINATION_PASSWORD"),
}

connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:

    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)

In [ ]:
#Функция удаления дубликатов
def remove_duplicates(data):
    feature_cols = data.columns.drop('customer_id').tolist()
    is_duplicated_features = data.duplicated(subset=feature_cols, keep=False)
    data = data[~is_duplicated_features].reset_index(drop=True)
    return data

In [ ]:
#Функция для заполнения пропусков
def fill_missing_values(data):
    cols_with_nans = data.isnull().sum()
    cols_with_nans = cols_with_nans[cols_with_nans > 0].index.drop('end_date')
    for col in cols_with_nans:
        if data[col].dtype in [float, int]:
            fill_value = data[col].mean()
        elif data[col].dtype == 'object':
            fill_value = data[col].mode().iloc[0]
        data[col] = data[col].fillna(fill_value)
    return data

In [ ]:
#Функция удаления выбросов
def remove_outliers(df: pd.DataFrame, threshold: float = 1.5) -> pd.DataFrame:
        num_cols = df.select_dtypes(include=['float']).columns
        potential_outliers = pd.DataFrame(False, index=df.index, columns=num_cols)
        
        for col in num_cols:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            margin = threshold * IQR
            lower = Q1 - margin
            upper = Q3 + margin
            potential_outliers[col] = ~df[col].between(lower, upper)
        
        outliers = potential_outliers.any(axis=1)
        df_cleaned = df[~outliers]
        return df_cleaned

In [ ]:
#Почистим датасет
df = fill_missing_values(df)
df = remove_duplicates(df)
df = remove_outliers(df)

In [ ]:
df = df.set_index('id')
df = df.drop(columns=['customer_id','begin_date','end_date'])

In [ ]:
obj_df = df.select_dtypes(include="object")
cat_columns = ["type", "payment_method", "internet_service", "gender"]

In [ ]:
num_df = df.select_dtypes(include='float64').dropna()
num_columns = ["monthly_charges", "total_charges"]

n_knots = 3
degree_spline = 4
n_quantiles=100
degree = 3
n_bins = 5
encode = 'ordinal'
strategy = 'uniform'
subsample = None

In [ ]:
df.dropna(subset=['monthly_charges','total_charges'],inplace=True)
num_df = df[num_columns]

In [ ]:
num_columns = ["monthly_charges", "total_charges"]

n_knots = 3
degree_spline = 4
n_quantiles=100
degree = 3
n_bins = 5
encode = 'ordinal'
strategy = 'uniform'
subsample = None


# SplineTransformer
encoder_spl = SplineTransformer(n_knots=n_knots, degree=degree_spline)
encoded_features = encoder_spl.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_spl.get_feature_names_out(num_columns))
num_df = pd.concat([num_df, encoded_df], axis=1)


# QuantileTransformer
encoder_q = QuantileTransformer(n_quantiles=n_quantiles)
encoded_features = encoder_q.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_q.get_feature_names_out(num_columns))

encoded_df.columns = [col + f"_q_{n_quantiles}" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)


# RobustScaler
encoder_rb = RobustScaler()
encoded_features = encoder_rb.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_rb.get_feature_names_out(num_columns))
encoded_df.columns = [col + f"_robust" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)


# PolynomialFeatures
encoder_pol = PolynomialFeatures(degree=degree)
encoded_features = encoder_pol.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_pol.get_feature_names_out(num_columns))
encoded_df.drop(encoded_df.columns[:1 + len(num_columns)], axis=1, inplace=True)
num_df = pd.concat([num_df, encoded_df], axis=1)

# KBinsDiscretizer
encoder_kbd = KBinsDiscretizer(n_bins=n_bins, encode=encode, strategy=strategy, subsample=subsample)
encoded_features = encoder_kbd.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_kbd.get_feature_names_out(num_columns))

encoded_df.columns = [col + f"_bin" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)


num_df.head(2)

In [ ]:
num_df

In [ ]:
numeric_transformer = ColumnTransformer(
    transformers=[
        ('spline', SplineTransformer(n_knots=n_knots, degree=degree_spline), num_columns),
        ('quantile', QuantileTransformer(n_quantiles=n_quantiles), num_columns),
        ('robust', RobustScaler(), num_columns),
        ('polynomial', PolynomialFeatures(degree=degree), num_columns),
        ('kbins', KBinsDiscretizer(n_bins=n_bins, encode=encode, strategy=strategy, subsample=subsample), num_columns),
    ]
)

categorical_transformer = Pipeline(
	steps=[
        ('onehot',OneHotEncoder(categories='auto', handle_unknown='ignore', max_categories=10, sparse_output=False, drop='first'))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
	('num',numeric_transformer,num_columns),
    ('cat',categorical_transformer,cat_columns)
    ]
)

encoded_features = preprocessor.fit_transform(df)

transformed_df = pd.DataFrame(encoded_features, columns=preprocessor.get_feature_names_out())

df = pd.concat([df.reset_index(drop=True), transformed_df.reset_index(drop=True)], axis=1)
df.head(2)

In [ ]:
#mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
#mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

#experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

#with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
#    run_id = run.info.run_id

#    mlflow.sklearn.log_model(preprocessor, "column_transformer") 
    
#    print(f"Run ID: {run_id}")

In [ ]:
# Обучение модели CatBoostClassifier
X = transformed_df
y = df['target']  # Целевая переменная

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = CatBoostClassifier(
    iterations=100, 
    depth=4, 
    learning_rate=0.1, 
    loss_function='Logloss', 
    verbose=0
)
model.fit(X_train, y_train)

In [ ]:
# Предсказания
y_pred = model.predict(X_test)  # Бинарные предсказания
y_proba = model.predict_proba(X_test)[:, 1]  # Вероятности положительного класса

In [ ]:
# Метрики
roc_auc = roc_auc_score(y_test, y_proba)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
logloss = log_loss(y_test, y_proba)

# Матрица ошибок с нормализацией
conf_matrix_normalized = confusion_matrix(y_test, y_pred, normalize='all')

# Ошибки первого и второго рода
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
err_1 = fp / (fp + tn)  # Ошибка первого рода
err_2 = fn / (fn + tp)  # Ошибка второго рода

In [ ]:
# поднимаем MLflow локально
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

# устанавливаем host, который будет отслеживать наши эксперименты
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

In [ ]:
# Настройки логирования модели
# ------------------------------------------------------------------
pip_requirements = "/home/mle-user/mle_projects/mle-mlflow/requirements.txt"
signature = mlflow.models.infer_signature(X_test, y_pred)
input_example = X_test[:10]
metadata = {'model_type': 'monthly'}

# Получим ID эксперимента
experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id
RUN_NAME = "model_6_registry"

In [ ]:
 #Логирование в MLflow
# ------------------------------------------------------------------
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    
    # Логируем гиперпараметры модели
    mlflow.log_param("iterations", 100)
    mlflow.log_param("depth", 4)
    mlflow.log_param("learning_rate", 0.1)
    mlflow.log_param("loss_function", "Logloss")
    
    # Логируем метрики
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("log_loss", logloss)
    mlflow.log_metric("error_type_1", err_1)
    mlflow.log_metric("error_type_2", err_2)

    # Пример: логируем матрицу ошибок как артефакт (картинку)
    plt.figure(figsize=(6, 6))
    sns.heatmap(conf_matrix_normalized, annot=True, fmt=".2f", cmap="Blues")
    plt.title("Normalized Confusion Matrix")
    plt.savefig("conf_matrix.png")
    plt.close()
    mlflow.log_artifact("conf_matrix.png")

    # Логируем модель (CatBoost) вместе с окружением
    model_info = mlflow.catboost.log_model(
        cb_model=model,
        artifact_path="models",
        registered_model_name=REGISTRY_MODEL_NAME,
        input_example=input_example,
        metadata=metadata,
        signature=signature,
        pip_requirements=pip_requirements,
        await_registration_for=60  # время ожидания регистрации в Model Registry
    )

print("Run ID:", run_id)
print("Model logged to MLflow with ID:", model_info.model_uri)
print(f"✅ Model registered successfully!")